In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

plt.rcParams["figure.figsize"] = (16,12)
plt.rcParams["font.size"] = 24
plt.rcParams['xtick.labelsize'] = "small"
plt.rcParams['ytick.labelsize'] = 36

In [25]:
# load the files
bead_date =20200518
bead_number = 2
dataset = "Shaking/Shaking375/"
FOI = "Shaking375_7_"
dirname ="/data/new_trap/" + str(bead_date) + "/Bead%s/" %bead_number +dataset
filename = dirname + FOI


In [26]:
s = time.time()
df_spin = spin_processor(bead_date,bead_number,dataset,FOI,1000)
e = time.time()
print(e-s)

  0%|          | 0/20 [00:00<?, ?it/s]

1000


100%|██████████| 20/20 [00:51<00:00,  2.30s/it]


60.71520233154297


In [27]:
# save the file
base_proc = "/data/new_trap_processed/processed_files/" + str(bead_date) +  "/Bead%s/" %bead_number

try:
    os.makedirs(base_proc+dataset)
    print("Created subdirs %s" %dataset)
except:
    print("Folder exists or you do not have permissions")
    
outputname = base_proc + dataset+FOI+"_spin.pkl"
if(os.path.isfile(outputname)==False):
    df_spin.to_pickle(outputname)
    print("Successfully saved spin file")
else:print("No spin file produced. %s exists already!" %outputname)    

Folder exists or you do not have permissions
Successfully saved spin file


In [28]:
del df_spin

In [29]:
base_proc = "/data/new_trap_processed/processed_files/" + str(bead_date) +  "/Bead%s/" %bead_number
df_spin = pd.read_pickle(base_proc + dataset+FOI+"_spin.pkl")

In [30]:
def freq_phase_extractor(df_spin,full=True):
    frequency = 3
    no_harmonics=15
    if(full==True): fq = np.arange(10,25000,10)
    if(full==False): fq= np.arange(frequency*10,no_harmonics*10*frequency,10*frequency)   
    for f in tqdm(fq):
        foi =[]
        poi=[]
        for i in range(len(df_spin)):
            foi.append(df_spin.spin_amp[i][f-5:f+5].mean())
            poi.append(df_spin.spin_phase[i][f])
        df_spin["freq_%d" %(f/10)] = foi
        df_spin["phase_%d"%(f/10)] = poi
    return df_spin

In [31]:
df_spin = freq_phase_extractor(df_spin,full=True)

100%|██████████| 2499/2499 [02:01<00:00, 20.34it/s]


In [32]:
base_proc = "/data/new_trap_processed/processed_files/" + str(bead_date) +  "/Bead%s/" %bead_number
outputname = base_proc + dataset+FOI+"_spin_full_freq_only.pkl"
df_red = df_spin.drop(columns=["spin_amp","spin_phase","spin_time"])
df_red.to_pickle(outputname)
print(outputname)

/data/new_trap_processed/processed_files/20200518/Bead2/Shaking/Shaking375/Shaking375_7__spin_full_freq_only.pkl
